## Day 7 

In [9]:
using DelimitedFiles

In [5]:
rndict = Dict(1 => 3, 2 => 3, 
              3 => 1, 4 => 1, 
              5 => 2, 6 => 2, 
              7 => 3, 8 => 3,
              99 => 0)

struct OpCode
    instr::Int
    mode::Array{Int, 1}
    N::Int
end

function OpCode(c::Int)
    digs = digits(c)
    oc = digs[1] + (length(digs) > 1 ? 10*digs[2] : 0)
    N = rndict[oc]
    mode = zeros(Int, N)
    mode[1:length(digs[3:end])] = digs[3:end]
    return OpCode(oc, mode, N)
end 


function intcode_computer(program, input::Channel{Int}, output::Channel{Int}; pc=1, name="A")
    jump_flag = false
    outs = []
    
    function load(dst)
        inp = take!(input)
        program[dst] = inp
    end
    
    function store(data)
        put!(output, data)    
    end    
    
    function jtrue(x, y)
        if x != 0 
            pc = y+1
            jump_flag = true
        end
    end
    
    function jfalse(x, y)
        if x == 0  
            pc = y+1 
            jump_flag = true
        end
    end
    
    lt(x, y) = Int(x < y)
    
    eq(x, y) = Int(x == y)
        
    
    while program[pc] != 99 && pc < length(program)
        jump_flag = false
        oc = OpCode(program[pc])     
        #println(oc)

        if oc.N == 3 || oc.N == 2
                    
            x, y = program[pc+1:pc+2]
            
            a = oc.mode[1] == 0 ? program[x+1] : x
            b = oc.mode[2] == 0 ? program[y+1] : y
            
            #println("A = $(a), B = $(b)")
                        
            if oc.N == 3
                r = program[pc+3]
                if oc.mode[3] == 1
                    error("Invalide modesetting for R register!")
                end
            #println("R = $(r)")
            end
            
            if oc.instr == 1
                program[r+1] = a+b
            elseif oc.instr == 2
                program[r+1] = a*b
            elseif oc.instr == 5
                jtrue(a, b)
            elseif oc.instr == 6
                jfalse(a, b)
            elseif oc.instr == 7
                program[r+1] = lt(a,b)
            elseif oc.instr == 8
                program[r+1] = eq(a,b)
            end
            
            
        elseif oc.N == 1
            x = program[pc+1]
            #println("X = $(x)")
            if oc.instr == 3
                load(x+1)
                #println("loaded to $(x+1)")
            elseif oc.instr == 4
                if oc.mode[1] == 1
                    store(x)
                else
                    store(program[x+1])
                end
            end
        else
            @error "Something went wrong! Got code $(oc.instr)."
        end
        pc += (jump_flag ? 0 : oc.N+1)
        #println("PC = $(pc)")
    end
    #println("Computer $(name) finished, output is $(outs[1])")
    #put!(output, outs[1])
    #println(output)
end

intcode_computer (generic function with 1 method)

In [6]:
function amplifiers_linear(phases, program)
    chans = [Channel{Int}(2) for x = 1:6]
    
    for j=1:5
        put!(chans[j], phases[j])
    end
    
    put!(chans[1], 0)
        
    @async intcode_computer(copy(program), chans[1], chans[2], name="A") #A
    @async intcode_computer(copy(program), chans[2], chans[3], name="B") #B
    @async intcode_computer(copy(program), chans[3], chans[4], name="C") #C
    @async intcode_computer(copy(program), chans[4], chans[5], name="D") #D
    @async intcode_computer(copy(program), chans[5], chans[6], name="E") #E
    
    return take!(chans[6])
end

amplifiers_linear (generic function with 1 method)

In [7]:
program = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
phases = [4,3,2,1,0]
amplifiers_linear(phases, program)

43210

In [10]:
using Combinatorics
phases = permutations([0,1,2,3,4])
program = readdlm("inputs/day7.txt", ',', Int)[:];

In [13]:
maximum([amplifiers_linear(p, program) for p in phases])

13848

In [14]:
function amplifiers_feedback(phases, program)
    chans = [Channel{Int}(2) for x = 1:5]
    
    for j=1:5
        put!(chans[j], phases[j])
    end
    
    put!(chans[1], 0)
    
    @sync begin
        @async intcode_computer(copy(program), chans[1], chans[2], name="A") #A
        @async intcode_computer(copy(program), chans[2], chans[3], name="B") #B
        @async intcode_computer(copy(program), chans[3], chans[4], name="C") #C
        @async intcode_computer(copy(program), chans[4], chans[5], name="D") #D
        @async intcode_computer(copy(program), chans[5], chans[1], name="E") #E
    end
    
    return take!(chans[1])
end

amplifiers_feedback (generic function with 1 method)

In [16]:
program = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
phases = [9,8,7,6,5]
amplifiers_feedback(phases, program)

139629729

In [17]:
program = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]
phases = [9,7,8,5,6]
amplifiers_feedback(phases, program)

18216

In [18]:
phases = permutations([5,6,7,8,9])
program = readdlm("inputs/day7.txt", ',', Int)[:];
maximum([amplifiers_feedback(p, program) for p in phases])

12932154